In [5]:
import numpy as np
import matplotlib.pylab as plt

# 6. 학습 관련 기술들

최적화, 초깃값, 하이퍼파라미터, 오버피팅 대응책(가중치 감소, 드롭아웃), 배치 정규화

## 6.1 매개변수 갱신

신경망 학습의 목적은 손실 함수의 값을 가능한 한 낮추는 매개변수를 찾는 것  
이는 곧 최적 매개변수를 찾는 문제이며, 이러한 문제를 푸는 것을 `최적화(optimization)`라고 함

### 6.1.1 모험가 이야기

광대하고 복잡한 지형을 지도도 없이 눈을 가린채로 '깊은 곳'을 찾지 않으면 안되는 상황에서  
중요한 단서가 되는 것이 땅의 `기울기`

### 6.1.2 확률적 경사 하강법(SGD)

수식으로 쓰면

$$ \mathbf{W} \leftarrow \mathbf{W} - \eta \frac{\partial{L}}{\partial{\mathbf{W}}} $$

In [6]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr

    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]

### 6.1.3 SGD의 단점

다음 함수의 최솟값을 구하는 문제를 생각해보자

$$ f(x,y) = \frac{1}{20}x^2 + y^2 $$

이 함수의 그래프와 등고선

<img width="648" alt="fig 6-1" src="https://user-images.githubusercontent.com/78716519/194982136-b6e76067-1ed4-409a-95b1-79f8f1c98424.png">

이 함수의 기울기를 그려보면

<img width="551" alt="fig 6-2" src="https://user-images.githubusercontent.com/78716519/194982284-f449d976-a6ca-4c88-87a3-511961e9734a.png">

이 기울기는 y축 방향은 크고, x축 방향은 작음  
즉 y축 방향은 가파른데, x축 방향은 완만해서  
최솟값이 되는 장소는 $(x,y) = (0,0)$이지만  
위의 그림이 보여주는 기울기 대부분은 $(0,0)$ 방향을 가리키지 않는다는 것

위의 함수에 SGD를 적용해보면

<img width="563" alt="fig 6-3" src="https://user-images.githubusercontent.com/78716519/194982516-2a938087-a225-441e-b9ef-a2862bf19aa9.png">

비효율적인 움직임 → SGD의 이러한 단점을 개선해주는 Momentum, AdaGrad, Adam 소개

### 6.1.4 모멘텀

`모멘텀(momentum)`: '운동량'을 뜻하는 단어

$$ \mathbf{v} \leftarrow \alpha \mathbf{v} - \eta \frac{\partial{L}}{\partial{\mathbf{W}}} $$
$$ \mathbf{W} \leftarrow \mathbf{W} + \mathbf{v} $$

$\mathbf{W}$: 갱신할 가중치 매개변수  
$\frac{\partial{L}}{\partial{\mathbf{W}}}$: $\mathbf{W}$에 대한 손실함수의 기울기  
$\eta$: 학습률  
$\mathbf{v}$: 물리에서 말하는 속도(velocity)  
$\alpha \mathbf{v}$항은 물체가 아무런 힘을 받지 않을 때 서서히 하강시키는 역할  
$\alpha$: 물리에서는 지면 마찰이나 공기 저항에 해당, $0.9$ 등의 값으로 설정함

첫번째 식은 기울기 방향으로 힘을 받아 물체가 가속된다는 물리 법칙을 나타냄  
모멘텀은 아래의 그림과 같이 공이 그릇의 바닥을 구르는 듯한 움직임을 보여줌

<img width="629" alt="fig 6-4" src="https://user-images.githubusercontent.com/78716519/194983373-7cfbcf3d-4f39-4667-9af5-e437fdfb93e5.png">

In [7]:
class Momentum:
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None # 초기화 때는 아무 값도 담지 않음
    
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val) # 매개변수와 같은 구조의 데이터를 딕셔너리 변수로 저장

        for key in params.keys():
            self.v[key] = self.momentum*self.v[key] - self.lr*grads[key]
            params[key] += self.v[key]

모멘텀을 사용해서 6.1.3의 함수의 최적화 문제를 풀어보자

<img width="557" alt="fig 6-5" src="https://user-images.githubusercontent.com/78716519/194984403-7a73bfa6-4e80-464e-93d9-375501962fc1.png">

모멘텀의 갱신 경로는 공이 그릇 바닥을 구르듯 움직임  
SGD와 비교하면 '지그재그 정도'가 덜함